In [2]:
import os
import re
from moviepy import VideoFileClip, concatenate_videoclips

def natural_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def combine_mp4s_with_chapters(folder_path, output_name="combined.mp4"):
    # Get all MP4 files in the folder, sorted alphabetically
    mp4_files = [f for f in os.listdir(folder_path) if f.lower().endswith('.mp4')]
    mp4_files.sort(key=natural_key)
    if not mp4_files:
        print("No MP4 files found in the folder.")
        return

    clips = []
    chapter_times = []
    current_time = 0
    for mp4 in mp4_files:
        clip = VideoFileClip(os.path.join(folder_path, mp4))
        clips.append(clip)
        chapter_times.append((current_time, os.path.splitext(mp4)[0]))
        current_time += clip.duration

    final_clip = concatenate_videoclips(clips, method="compose")
    output_path = os.path.join(folder_path, output_name)
    final_clip.write_videofile(output_path, codec="libx264", audio_codec="aac")

    # Write chapters file in ffmetadata format
    chapters_path = os.path.join(folder_path, "chapters.txt")
    with open(chapters_path, "w", encoding="utf-8") as f:
        f.write(";FFMETADATA1\n")
        for i, (start, title) in enumerate(chapter_times):
            if i + 1 < len(chapter_times):
                end = int(chapter_times[i+1][0])
            else:
                end = int(current_time)
            f.write("[CHAPTER]\n")
            f.write("TIMEBASE=1/1\n")
            f.write(f"START={int(start)}\n")
            f.write(f"END={end}\n")
            f.write(f"title={title}\n")

    print(f"Combined video saved to: {output_path}")
    print(f"Chapters file saved to: {chapters_path}")
    print("Now run the following ffmpeg command to add chapters:")

    print(f'ffmpeg -i "{output_path}" -i "{chapters_path}" -map_metadata 1 -codec copy "{output_path[:-4]}_with_chapters.mp4"')

# Example usage:
# combine_mp4s_with_chapters(r"C:\path\to\your\mp4_folder")

In [ ]:
combine_mp4s_with_chapters(r"Z:\OneDrive\Gardening_2025\Strategy_MIT\Module_9 - Developing a New Leadership Mindset for Data\videos")

MoviePy - Building video Z:\OneDrive\Gardening_2025\Strategy_MIT\Module_9 - Developing a New Leadership Mindset for Data\videos\combined.mp4.
MoviePy - Writing audio in combinedTEMP_MPY_wvf_snd.mp4


chunk:  32%|███▏      | 35621/110837 [02:14<15:12, 82.42it/s, now=None] 